# TemplateBank

A TemplateBank is a thin wrapper on an [obsplus EventBank](https://niosh-mining.github.io/obsplus/notebooks/interfaces/eventbank.html).  TemplateBanks handle database tasks for storing, selecting and reading in catalogues of templates from/on disk. The idea is to have a suite of templates for all possible events of interest; when something interesting happens this database can be queried to provide the relevant templates for that trigger. Because `TemplateBank`s subclass `EventBank`, all the look-up methods for `EventBank` are available.

Lets have a look at how we might generate a `TemplateBank` from scratch for a day of the New Zealand national catalogue.

In [1]:
from rt_eqcorrscan.database import TemplateBank
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

bank = TemplateBank(base_path="./template_bank")
client = Client("GEONET")
catalog = client.get_events(
    starttime=UTCDateTime(2019, 6, 21),
    endtime=UTCDateTime(2019, 6, 23),
    latitude=-38.8, longitude=175.8, maxradius=0.2)
print(catalog)

7 Event(s) in Catalog:
2019-06-21T03:46:25.054642Z | -38.824, +175.865 | 2.178845358 M  | manual
2019-06-21T03:52:00.142063Z | -38.826, +175.855 | 1.230595585 M  | manual
2019-06-21T08:28:16.143864Z | -38.827, +175.866 | 1.364494767 M  | manual
2019-06-21T08:29:23.083982Z | -38.827, +175.878 | 2.066853896 M  | manual
2019-06-21T09:07:54.730731Z | -38.819, +175.904 | 1.420446524 M  | manual
2019-06-21T15:34:49.878441Z | -38.764, +175.615 | 1.530488164 M  | manual
2019-06-22T09:35:27.682975Z | -38.819, +175.866 | 1.880393466 M  | manual


First we will put the events into the database.

In [2]:
bank.put_events(catalog)
bank.read_index()

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,path
0,2019-06-21 03:46:25.054642,-38.823746,175.864609,5000.000000,2.178845,Taupo,46.0,67.671886,smi:nz.org.geonet/2019p462586,3811.082605,...,0.279893,25.0,39.0,0.000000,2020-04-29 04:26:43.090682624,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-21 03:46:42.845267,,/2019/06/21/2019-06-21T03-46-25_62586.xml
1,2019-06-21 03:52:00.142063,-38.826271,175.855438,5000.000000,1.230596,Taupo,21.0,163.628212,smi:nz.org.geonet/2019p462596,6603.341442,...,0.229197,15.0,15.0,0.000000,2020-04-29 04:26:43.098682624,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-21 03:52:21.600176,,/2019/06/21/2019-06-21T03-52-00_62596.xml
2,2019-06-21 08:28:16.143864,-38.827057,175.866028,7828.764915,1.364495,Taupo,26.0,91.194962,smi:nz.org.geonet/2019p463116,5572.845242,...,0.255632,16.0,19.0,8385.635050,2020-04-29 04:26:43.106682624,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-21 08:28:36.574978,,/2019/06/21/2019-06-21T08-28-16_63116.xml
3,2019-06-21 08:29:23.083982,-38.827339,175.877670,5000.000000,2.066854,Taupo,61.0,40.233490,smi:nz.org.geonet/2019p463118,3076.595177,...,0.350769,37.0,53.0,0.000000,2020-04-29 04:26:43.130682368,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-21 08:29:41.755653,,/2019/06/21/2019-06-21T08-29-23_63118.xml
4,2019-06-21 09:07:54.730731,-38.819088,175.904159,5000.000000,1.420447,Taupo,32.0,76.976382,smi:nz.org.geonet/2019p463190,4347.700016,...,0.206139,18.0,28.0,0.000000,2020-04-29 04:26:43.142682624,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-21 09:08:14.577520,,/2019/06/21/2019-06-21T09-07-54_63190.xml
5,2019-06-21 15:34:49.878441,-38.764454,175.615082,113987.533600,1.530488,Taupo,21.0,265.354118,smi:nz.org.geonet/2019p463916,11605.852290,...,0.406860,21.0,14.0,13899.074190,2020-04-29 04:26:43.146682624,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-21 15:35:31.012134,,/2019/06/21/2019-06-21T15-34-49_63916.xml
6,2019-06-22 09:35:27.682975,-38.818531,175.866409,13595.767970,1.880393,Taupo,39.0,56.649483,smi:nz.org.geonet/2019p465944,5168.447127,...,0.478102,26.0,34.0,7484.634962,2020-04-29 04:26:43.162682368,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),2019-06-22 09:35:48.159662,,/2019/06/22/2019-06-22T09-35-27_65944.xml


Now we want to populate the database with templates: at the moment the database only contains event
files. Note that this might take a while - it will download the required data and process it.  
The same arguments used by EQcorrscan's [Tribe.construct](https://eqcorrscan.readthedocs.io/en/latest/submodules/core.match_filter.Tribe.html#eqcorrscan.core.match_filter.Tribe.construct) 
are supported by `Template_bank.make_templates`.

In [3]:
bank.make_templates(
    catalog=catalog, client=client, lowcut=2., highcut=15.,
    samp_rate=50., filt_order=4, prepick=0.5, length=3, swin="all")

No data downloaded for NZ.NNVZ.10.EHT
No data downloaded for NZ.BKZ.10.HHR
No data downloaded for NZ.BKZ.10.HHT
No data downloaded for NZ.KWHZ.10.EHR
No data downloaded for NZ.MTHZ.10.EHR
No data downloaded for NZ.HIZ.10.HHT
No data downloaded for NZ.HIZ.10.HHT
No data downloaded for NZ.TSZ.10.HHR
No data downloaded for NZ.RITZ.10.EHT
No data downloaded for NZ.WATZ.10.EHT
No data downloaded for NZ.WHTZ.10.EHT
No data downloaded for NZ.KATZ.10.EHT
No data downloaded for NZ.TMVZ.10.HHT
No data downloaded for NZ.ETVZ.10.HHR
No data downloaded for NZ.ETVZ.10.HHR
No data downloaded for NZ.NNVZ.10.EHT
No data downloaded for NZ.OTVZ.10.HHT
No data downloaded for NZ.SNVZ.10.EHT
No data downloaded for NZ.MRHZ.10.EHR
Request would result in too much data. Denied by the datacenter. Split the request in smaller parts
Detailed response of server:


/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/io/mseed/core.py:790: UserWarning: The encoding specified in trace.stats.mse

Tribe of 7 templates

This both returns the `Tribe` of templates that are created, and writes them to disk for later retrieval.
Note that lots of warnings are raised by this due to missing links between picks. These can be
ignored for this tutorial.

In [4]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tribe = bank.get_templates()
print(tribe)

Tribe of 7 templates


`get_templates` supports filtering using event queries:

In [5]:
bank.get_templates(starttime=UTCDateTime(2019, 6, 21, 9))

Tribe of 3 templates

## Usage in RT-EQcorrscan

A `TemplateBank` is required to use the `CatalogListener` and `Reactor` classes. Both these classes will *listen* to a remote client service for new events and add those events to the `TemplateBank`. The `Reactor` class will also react to any events that meet user-defined thresholds, collect the appropriate temlates using the `TemplateBank`, and run the real-time matched-filter process.

## Usage outside of RT-EQcorrscan

Because `TemplateBanks` return `Tribe`s, `TemplateBanks` represent an efficient means of storing `Tribes` on disk.  `Tribe`s can be used offline for matched-filter detection.